# Downloading tcg datas

### Imports & Setup the environment

In [190]:
# Made on python 3.12

import pandas as pd #library for data analysis
import numpy as np #library for scientific computing with Python
import matplotlib.pyplot as plt #library for creating static, animated, and interactive visualizations
from matplotlib.image import imread #library for reading and writing image files
import cv2 #Manipulate images and videos,
import os #manage files and directories
from PIL import Image ,ImageOps # Python Imaging Library adds image processing capabilities to your Python interpreter. 
from dotenv import load_dotenv #load environment variables from .env file
import math #mathematical functions
import json #library for working with JSON
import requests #library for making HTTP requests
import shutil #library for copying and moving files
import io
import urllib.request
import base64
from imageio import imread
import random


load_dotenv()


True

### Global variables

In [7]:
API_NINJA_API_KEY = os.getenv("API_NINJA_API_KEY")
TCG_POKEMON_API_KEY = os.getenv("TCG_POKEMON_API_KEY")
TCG_POKEMON_MAX_CARDS_EXISTING = 19501
TCG_POKEMON_MAX_POKEMON_PER_PAGE = 250
DATA_TCG_FOLDER_PATH = "data/tcg/"
GENERATED_DATAS_V1_FOLDER_PATH = "data/generated_datas_v1/"
GENERATED_DATAS_V1_IMAGES_FOLDER_PATH = GENERATED_DATAS_V1_FOLDER_PATH + "images/"
GENERATED_DATAS_V1_JSON_FOLDER_PATH = GENERATED_DATAS_V1_FOLDER_PATH + "json/"


### Configuration

In [9]:
if not os.path.exists(GENERATED_DATAS_V1_FOLDER_PATH):
    os.makedirs(GENERATED_DATAS_V1_FOLDER_PATH)
    print(GENERATED_DATAS_V1_FOLDER_PATH +" folder created")

if not os.path.exists(GENERATED_DATAS_V1_IMAGES_FOLDER_PATH):
    os.makedirs(GENERATED_DATAS_V1_IMAGES_FOLDER_PATH)
    print(GENERATED_DATAS_V1_IMAGES_FOLDER_PATH +" folder created")

if not os.path.exists(GENERATED_DATAS_V1_JSON_FOLDER_PATH):
    os.makedirs(GENERATED_DATAS_V1_JSON_FOLDER_PATH)
    print(GENERATED_DATAS_V1_JSON_FOLDER_PATH +" folder created")

### Load a random image for the background

In [220]:

def load_random_image(**kwargs):
    RESIZE_SIZES = kwargs.get("RESIZE_SIZES")
    RGB_COLOR = kwargs.get("RGB_COLOR")
    if RGB_COLOR == None:
        RGB_COLOR = True
    response = requests.get("https://api.api-ninjas.com/v1/randomimage",headers={"X-Api-Key": API_NINJA_API_KEY, 'Accept': 'image/jpg'})
    image = Image.open(BytesIO(response.content))
    if RESIZE_SIZES:
        image = image.resize(RESIZE_SIZES)
    if RGB_COLOR == False:
        image = ImageOps.grayscale(image)
    return image




In [217]:
TCG_CARDS_IDS_LIST = os.listdir(DATA_TCG_FOLDER_PATH)

def load_tcg_data(**kwargs):
    card_id = kwargs.get("card_id")
    if card_id == None:
        card_id = random.choice(TCG_CARDS_IDS_LIST)
    card_id = card_id.replace(".json", "")

    with open(DATA_TCG_FOLDER_PATH + card_id + ".json") as f:
        card_data = json.load(f)
    return card_data

def load_tcg_image(tcg_data,**kwargs):
    RESIZE_SIZES = kwargs.get("RESIZE_SIZES")
    RGB_COLOR = kwargs.get("RGB_COLOR")
    if RGB_COLOR == None:
        RGB_COLOR = True
    response = requests.get(tcg_data["images"]["large"],headers={"X-Api-Key": TCG_POKEMON_API_KEY, 'Accept': 'image/jpg'})
    image = Image.open(BytesIO(response.content))
    if RESIZE_SIZES:
        image = image.resize(RESIZE_SIZES)
    if RGB_COLOR == False:
        image = ImageOps.grayscale(image)
    return image



In [320]:

def apply_skew_effect(image, top_left_skew=(0, 0), top_right_skew=(0, 0), bottom_left_skew=(0, 0), bottom_right_skew=(0, 0)):
    
    # Convertir PIL en OpenCV si nécessaire
    if hasattr(image, 'size'):  # C'est une image PIL
        print("Image PIL détectée, conversion en OpenCV...")
        # Convertir PIL en array numpy
        img_array = np.array(image)
        # Convertir RGB en BGR pour OpenCV
        if len(img_array.shape) == 3:
            img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        image = img_array
    
    # Maintenant on a un array OpenCV
    if len(image.shape) == 3:
        height, width, _ = image.shape
    else:
        height, width = image.shape

    print(f"Dimensions: {height} x {width}")
    
    # Points d'origine (coins de l'image rectangulaire)
    src_points = np.float32([
        [0, 0],           # top-left
        [width-1, 0],     # top-right
        [0, height-1],    # bottom-left
        [width-1, height-1] # bottom-right
    ])
    
    # Points de destination (avec déformation personnalisée)
    dst_points = np.float32([
        # Top-left: position originale + décalage (x, y)
        [0 + top_left_skew[0], 0 + top_left_skew[1]],
        # Top-right: position originale + décalage (x, y)
        [width-1 + top_right_skew[0], 0 + top_right_skew[1]],
        # Bottom-left: position originale + décalage (x, y)
        [0 + bottom_left_skew[0], height-1 + bottom_left_skew[1]],
        # Bottom-right: position originale + décalage (x, y)
        [width-1 + bottom_right_skew[0], height-1 + bottom_right_skew[1]]
    ])
    
    # Calculer la matrice de transformation perspective
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    
    # Appliquer la transformation
    skewed_image = cv2.warpPerspective(image, matrix, (width, height))
    
    # Préparer les coordonnées finales des angles
    final_coordinates = {
        'top_left': (int(dst_points[0][0]), int(dst_points[0][1])),
        'top_right': (int(dst_points[1][0]), int(dst_points[1][1])),
        'bottom_left': (int(dst_points[2][0]), int(dst_points[2][1])),
        'bottom_right': (int(dst_points[3][0]), int(dst_points[3][1]))
    }
    
    print("Coordonnées finales des angles:")
    for corner, coords in final_coordinates.items():
        print(f"  {corner}: {coords}")
    
    return skewed_image, final_coordinates


In [321]:
random_image = load_random_image(RESIZE_SIZES=(1200,1500))
print(type(random_image))

tcg_datas = load_tcg_data()
tcg_image = load_tcg_image(tcg_datas)
print(type(tcg_image))


result1, coords1 = apply_skew_effect(
    tcg_image.copy(),
    top_left_skew=(100, 20),    # Décaler vers la droite de 30px, vers le bas de 20px
    top_right_skew=(-20, 100),  # Décaler vers la gauche de 15px, vers le bas de 25px
    bottom_left_skew=(20, -100), # Décaler vers la droite de 20px, vers le haut de 10px
    bottom_right_skew=(-100, -20) # Décaler vers la gauche de 40px, vers le haut de 30px
)
print(type(result1))
print(coords1)

tmp = cv2.cvtColor(result1, cv2.COLOR_BGR2GRAY)
_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
b, g, r = cv2.split(result1)
rgba = [b,g,r, alpha]
dst = cv2.merge(rgba,4)
pil_image = Image.fromarray(dst)
random_image.paste(pil_image, (0,0),pil_image)
cv2.imwrite(f"test.jpg", np.array(random_image))



<class 'PIL.Image.Image'>
<class 'PIL.PngImagePlugin.PngImageFile'>
Image PIL détectée, conversion en OpenCV...
Dimensions: 1024 x 734
Coordonnées finales des angles:
  top_left: (100, 20)
  top_right: (713, 100)
  bottom_left: (20, 923)
  bottom_right: (633, 1003)
<class 'numpy.ndarray'>
{'top_left': (100, 20), 'top_right': (713, 100), 'bottom_left': (20, 923), 'bottom_right': (633, 1003)}


True